In [ ]:
scoring_metrics = [
    "FGM", "FGA", "FG3M", "FG3A", "FTM", "FTA", "PTS", "PLUS_MINUS", 
    "FGM_PG", "FGA_PG", "ITP_FGM", "ABOVE_BREAK_3_FGA", "MID_FGM", 
    "RIGHT_CORNER_3_FGM", "MID_FGA", "RA_FGA", "LEFT_CORNER_3_FGM", 
    "ITP_FGA", "LEFT_CORNER_3_FGA", "BACKCOURT_FGA", "RA_FGM", 
    "CORNER_3_FGM", "RIGHT_CORNER_3_FGA", "BACKCOURT_FGM", 
    "CORNER_3_FGA", "ABOVE_BREAK_3_FGM", "PULL_UP_PTS", 
    "PAINT_TOUCH_PTS", "ELBOW_TOUCH_PTS", "POST_TOUCH_PTS", 
    "CATCH_SHOOT_PTS"
]

shooting_zones = [
    "FGM_10_14", "FGP_5_9", "FGP_35_39", "FGP_LT_5", "FGP_10_14", 
    "FGA_15_19", "FGP_15_19", "FGA_25_29", "FGM_40_PLUS", 
    "FGM_LT_5", "FGM_15_19", "FGA_40_PLUS", "FGM_20_24", 
    "FGA_30_34", "FGM_5_9", "FGM_30_34", "FGA_10_14", "FGP_30_34", 
    "FGA_5_9", "FGP_25_29", "FGP_40_PLUS", "FGM_25_29", 
    "FGA_20_24", "FGP_20_24", "FGA_35_39", "FGM_35_39", "FGA_LT_5"
]

passing_and_assists = [
    "AST", "AST_RATIO", "AST_TO", "POTENTIAL_AST", "AST_ADJ", 
    "FT_AST", "PASSES_RECEIVED", "PASSES_MADE", "SECONDARY_AST", 
    "AST_PTS_CREATED", "AST_TO_PASS_PCT", "AST_TO_PASS_PCT_ADJ"
]

rebounds = [
    "OREB", "DREB", "REB", "REB_CHANCES", "REB_CONTEST", 
    "OREB_CHANCES", "OREB_CONTEST", "DREB_CHANCES", "DREB_CONTEST", 
    "REB_CHANCE_DEFER", "DREB_CHANCE_DEFER", "OREB_CHANCE_DEFER", 
    "AVG_REB_DIST", "AVG_OREB_DIST", "AVG_DREB_DIST", 
    "REB_UNCONTEST", "OREB_UNCONTEST", "DREB_UNCONTEST"
]

defense = [
    "STL", "BLK", "BLKA", "PF", "PFD", "sp_work_DEF_RATING", 
    "DEF_RATING", "E_DEF_RATING", "DREB_PCT", "OREB_PCT", 
    "REB_PCT", "OREB_CHANCE_PCT", "DREB_CHANCE_PCT", 
    "OREB_CHANCE_PCT_ADJ", "DREB_CHANCE_PCT_ADJ", "REB_CHANCE_PCT", 
    "REB_CHANCE_PCT_ADJ", "REB_CONTEST_PCT", "DREB_CONTEST_PCT", 
    "OREB_CONTEST_PCT"
]

advanced_metrics = [
    "sp_work_NET_RATING", "NET_RATING", "E_NET_RATING", 
    "OFF_RATING", "E_OFF_RATING", "PACE", "PACE_PER40", "E_PACE", 
    "TS_PCT", "EFG_PCT", "EFF_FG_PCT", "USG_PCT", "E_USG_PCT", 
    "TM_TOV_PCT", "E_TOV_PCT", "W_PCT", "PLUSMINUS", "DD2", "TD3"
]

shooting_accuracy = [
    "FG_PCT", "FG3_PCT", "FT_PCT", "MID_FG_PCT", 
    "BACKCOURT_FG_PCT", "ABOVE_BREAK_3_FG_PCT", "CORNER_3_FG_PCT", 
    "RA_FG_PCT", "LEFT_CORNER_3_FG_PCT", "RIGHT_CORNER_3_FG_PCT", 
    "ITP_FG_PCT", "very_tight_EFG_PCT", "very_tight_FG2_PCT", 
    "very_tight_FG3_PCT", "very_tight_FG_PCT", "tight_FG_PCT", 
    "tight_EFG_PCT", "tight_FG2_PCT", "tight_FG3_PCT", 
    "open_FG2_PCT", "open_EFG_PCT", "open_FG3_PCT", "open_FG_PCT", 
    "wide_open_FG2_PCT", "wide_open_FG_PCT", "wide_open_FG3_PCT", 
    "wide_open_EFG_PCT", "PULL_UP_EFG_PCT", "PULL_UP_FG3_PCT", 
    "PULL_UP_FG_PCT", "CATCH_SHOOT_FG_PCT", "PAINT_TOUCH_FG_PCT", 
    "ELBOW_TOUCH_FG_PCT", "POST_TOUCH_FG_PCT", "LT_06_PCT", 
    "NS_LT_06_PCT"
]

play_types = [
    "DRIVES", "DRIVE_FGM", "DRIVE_FGA", "DRIVE_FTM", "DRIVE_FTA", 
    "DRIVE_TOV", "DRIVE_PF", "DRIVE_AST", "DRIVE_PASSES", "DRIVE_PTS", 
    "DRIVE_FG_PCT", "DRIVE_PTS_PCT", "DRIVE_AST_PCT", "DRIVE_TOV_PCT", 
    "DRIVE_FT_PCT", "DRIVE_PASSES_PCT", "DRIVE_PF_PCT"
]

defensive_coverage = [
    "very_tight_FGM", "very_tight_FGA", "very_tight_FG3M", 
    "very_tight_FG3A", "very_tight_FG2M", "very_tight_FG2A", 
    "very_tight_FG2A_FREQUENCY", "very_tight_FGA_FREQUENCY", 
    "very_tight_FG3A_FREQUENCY", "tight_FGM", "tight_FGA", 
    "tight_FG3M", "tight_FG3A", "tight_FG2M", "tight_FG2A", 
    "tight_FG2A_FREQUENCY", "tight_FGA_FREQUENCY", 
    "tight_FG3A_FREQUENCY", "open_FGM", "open_FGA", "open_FG3M", 
    "open_FG3A", "open_FG2M", "open_FG2A", "open_FGA_FREQUENCY", 
    "open_FG3A_FREQUENCY", "open_FG2A_FREQUENCY", 
    "wide_open_FGM", "wide_open_FGA", "wide_open_FG3M", 
    "wide_open_FG3A", "wide_open_FG2M", "wide_open_FG2A", 
    "wide_open_FGA_FREQUENCY", "wide_open_FG3A_FREQUENCY", 
    "wide_open_FG2A_FREQUENCY"
]

touches_and_possession = [
    "TOUCHES", "AVG_SEC_PER_TOUCH", "TIME_OF_POSS", 
    "AVG_DRIB_PER_TOUCH", "PAINT_TOUCHES", "FRONT_CT_TOUCHES", 
    "ELBOW_TOUCHES", "POST_TOUCHES", "PTS_PER_ELBOW_TOUCH", 
    "PTS_PER_PAINT_TOUCH", "PTS_PER_TOUCH", "PTS_PER_POST_TOUCH"
]
